# 利用Python对链家网北京主城区二手房进行数据分析
* 本文主要讲述如何通过pandas对爬虫下来的链家数据进行相应的二手房数据分析，主要分析内容包括各个行政区，各个小区的房源信息情况。
* 数据来源 https://github.com/XuefengHuang/lianjia-scrawler 该repo提供了python程序进行链家网爬虫，并从中提取二手房价格、面积、户型和二手房关注度等数据。
* 分析方法参考 http://www.jianshu.com/p/44f261a62c0f

## 导入链家网二手房在售房源的文件（数据更新时间2017-11-29）

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import sys

stdout = sys.stdout

import importlib
importlib.reload(sys)
%matplotlib inline

#reload(sys)
#sys.setdefaultencoding('utf-8')
sys.stdout = stdout

plt.rcParams['font.sans-serif'] = ['SimHei']    
plt.rcParams['axes.unicode_minus'] = False

#所有在售房源信息
rentInfo=pd.read_csv('rentinfo.csv')

# 所有小区信息
community=pd.read_csv('community.csv')

# 合并小区信息和房源信息表，可以获得房源更详细的地理位置
community['community'] = community['title']
rent_house_detail = pd.merge(rentInfo, community, left_on='region', right_on='community')

In [4]:
rent_house_detail.shape

(9276, 31)

In [5]:
rent_house_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9276 entries, 0 to 9275
Data columns (total 31 columns):
houseID         9276 non-null object
title_x         9276 non-null object
link_x          9276 non-null object
region          9276 non-null object
zone            9276 non-null object
meters          9276 non-null object
other           9276 non-null object
subway          4375 non-null object
decoration      7257 non-null object
heating         9086 non-null object
price_x         9276 non-null int64
pricepre        9276 non-null object
updatedate      9276 non-null object
id              9276 non-null int64
title_y         9276 non-null object
link_y          9276 non-null object
district        9276 non-null object
bizcircle       9276 non-null object
tagList         6440 non-null object
onsale          9276 non-null int64
onrent          9276 non-null int64
year            9170 non-null object
housetype       9170 non-null object
cost            9170 non-null object
service  

In [11]:
rent_house_detail['zone'].value_counts()

3室1厅     2590
2室1厅     2528
1室1厅     1502
4室1厅      862
3室2厅      467
1室0厅      287
2室2厅      236
4室2厅      222
1房间1卫     106
5室1厅      102
2房间1卫      85
5室2厅       41
6室1厅       37
3房间1卫      36
3房间2卫      30
2室0厅       22
5室3厅       16
1室2厅       13
4室3厅       11
4房间2卫      10
3室0厅        9
1房间0卫       9
6室2厅        7
3室3厅        7
2房间2卫       5
5房间2卫       5
4房间1卫       3
6房间4卫       3
5房间1卫       3
5房间3卫       2
5室4厅        2
6室3厅        2
7房间2卫       2
6房间2卫       2
7室3厅        1
9室5厅        1
8室5厅        1
8室2厅        1
6房间1卫       1
2房间0卫       1
8房间2卫       1
4室0厅        1
1房间2卫       1
6室0厅        1
5房间4卫       1
4房间3卫       1
Name: zone, dtype: int64

In [43]:
dfzone = rent_house_detail[rent_house_detail['zone'].str.contains('卫')]
dfzone

,houseID,title_x,link_x,region,zone,meters,other,subway,decoration,heating,...,year,housetype,cost,service,company,building_num,house_num,price_y,validdate,community
177,101101887386,女人街 莱太花卉商场 华远九都汇 U.S. Embassy华远九都汇 5房间1卫 249...,https://bj.lianjia.com/zufang/101101887386.html,华远九都汇,5房间1卫,249平米,燕莎租房/低楼层(共4层)/2011年建板楼,NaN,精装修,集中供暖,...,2011年建成,板楼,6.99元/平米/月,北京圣瑞物业管理有限公司,北京金秋莱太房地产开发有限公司,6栋,502户,77583,2018-06-11 19:23:39,华远九都汇
179,101102601709,燕莎女人街，华远可注册办公，出行便利华远九都汇 3房间1卫 108平米 东北燕莎租房/...,https://bj.lianjia.com/zufang/101102601709.html,华远九都汇,3房间1卫,108平米,燕莎租房/低楼层(共7层)/2011年建板楼,NaN,精装修,集中供暖,...,2011年建成,板楼,6.99元/平米/月,北京圣瑞物业管理有限公司,北京金秋莱太房地产开发有限公司,6栋,502户,77583,2018-06-11 19:23:39,华远九都汇
181,101102794203,华远九都汇 3房间1卫 19500元华远九都汇 3房间1卫 114平米 南燕莎租房/中...,https://bj.lianjia.com/zufang/101102794203.html,华远九都汇,3房间1卫,114平米,燕莎租房/中楼层(共7层)/2011年建板楼,NaN,NaN,集中供暖,...,2011年建成,板楼,6.99元/平米/月,北京圣瑞物业管理有限公司,北京金秋莱太房地产开发有限公司,6栋,502户,77583,2018-06-11 19:23:39,华远九都汇
192,101102827921,林肯公园 3房间1卫 6500元林肯公园 3房间1卫 67平米 东北亦庄租房/高楼层(...,https://bj.lianjia.com/zufang/101102827921.html,林肯公园,3房间1卫,67平米,亦庄租房/高楼层(共24层)/2009年建塔楼,距离亦庄线亦庄文化园站368米,精装修,集中供暖,...,2009年建成,塔楼/板楼,2.28至4.5元/平米/月,北京晟邦物业管理有限公司,北京和裕房地产开发有限公司,29栋,4543户,67546,2018-06-11 20:07:27,林肯公园
193,101102850220,林肯公园精装修，大开间，随时签约,https://bj.lianjia.com/zufang/101102850220.html,林肯公园,1房间1卫,44平米,亦庄租房/低楼层(共24层)/2009年建塔楼,距离亦庄线亦庄文化园站204米,精装修,集中供暖,...,2009年建成,塔楼/板楼,2.28至4.5元/平米/月,北京晟邦物业管理有限公司,北京和裕房地产开发有限公司,29栋,4543户,67546,2018-06-11 20:07:27,林肯公园
194,101102863783,林肯公园一期4+1户型，精装修4室！林肯公园 5房间4卫 250平米 西南亦庄租房/中...,https://bj.lianjia.com/zufang/101102863783.html,林肯公园,5房间4卫,250平米,亦庄租房/中楼层(共24层)/2009年建塔楼,距离亦庄线亦庄文化园站454米,精装修,集中供暖,...,2009年建成,塔楼/板楼,2.28至4.5元/平米/月,北京晟邦物业管理有限公司,北京和裕房地产开发有限公司,29栋,4543户,67546,2018-06-11 20:07:27,林肯公园
195,101102870255,林肯公园 4房间3卫 24000元林肯公园 4房间3卫 220平米 东北亦庄租房/低楼...,https://bj.lianjia.com/zufang/101102870255.html,林肯公园,4房间3卫,220平米,亦庄租房/低楼层(共24层)/2009年建塔楼,NaN,精装修,集中供暖,...,2009年建成,塔楼/板楼,2.28至4.5元/平米/月,北京晟邦物业管理有限公司,北京和裕房地产开发有限公司,29栋,4543户,67546,2018-06-11 20:07:27,林肯公园
196,101102882314,林肯公园 2房间1卫 5100元林肯公园 2房间1卫 51平米 南亦庄租房/低楼层(共...,https://bj.lianjia.com/zufang/101102882314.html,林肯公园,2房间1卫,51平米,亦庄租房/低楼层(共24层)/2009年建塔楼,距离亦庄线亦庄文化园站631米,精装修,集中供暖,...,2009年建成,塔楼/板楼,2.28至4.5元/平米/月,北京晟邦物业管理有限公司,北京和裕房地产开发有限公司,29栋,4543户,67546,2018-06-11 20:07:27,林肯公园
197,101102922915,林肯公园 1房间1卫 4200元林肯公园 1房间1卫 35平米 北亦庄租房/高楼层(共...,https://bj.lianjia.com/zufang/101102922915.html,林肯公园,1房间1卫,35平米,亦庄租房/高楼层(共24层)/2011年建塔楼,NaN,NaN,集中供暖,...,2009年建成,塔楼/板楼,2.28至4.5元/平米/月,北京晟邦物业管理有限公司,北京和裕房地产开发有限公司,29栋,4543户,67546,2018-06-11 20:07:27,林肯公园
198,101102924026,林肯公园 2房间1卫 5000元林肯公园 2房间1卫 55平米 北亦庄租房/低楼层(共...,https://bj.lianjia.com/zufang/101102924026.html,林肯公园,2房间1卫,55平米,亦庄租房/低楼层(共24层)/2009年建塔楼,距离亦庄线亦庄文化园站368米,精装修,集中供暖,...,2009年建成,塔楼/板楼,2.28至4.5元/平米/月,北京晟邦物业管理有限公司,北京和裕房地产开发有限公司,29栋,4543户,67546,2018-06-11 20:07:27,林肯公园


In [ ]:
rent_house_detail[rent_house_detail.isnull().any(axis=1)].head(1)

In [ ]:
rent_house_detail.loc[rent_house_detail.isnull().any(axis=1),['houseID','subway']].head(2)

In [ ]:
nulls_dropped = rent_house_detail.dropna()
nulls_dropped.info()

In [ ]:
nulls_dropped[200:205][['houseID','subway']]

In [ ]:
priceX=nulls_dropped['price_x'] 
q1=priceX.quantile(.25)
q3=priceX.quantile(.75)
iqr = q3 - q1
pmin = q1 - 1.5*iqr
pmax = q3 + 1.5*iqr
nprice = priceX.where(priceX.between(pmin,pmax))
compare = pd.DataFrame({'before':priceX,'after':nprice})
compare.plot.box()
compare.describe()

In [ ]:
nulls_dropped.duplicated().any()

In [ ]:
df = pd.DataFrame({'sin': np.arange(0,5*np.pi,0.01),
                   'cos': np.arange(0.5*np.pi,5.5*np.pi,0.01)})
df.plot()

In [ ]:
nulls_dropped_thresh = rent_house_detail.dropna(thresh=3)
nulls_dropped_thresh.info()

In [ ]:
rent_house_detail.iloc[:,10].describe()

In [ ]:
rent_house_detail.iloc[:,10].value_counts()

In [ ]:
rent_house_detail[4:5][['title_x']]
rent_house_detail.loc[4:5,'title_x']

## 将数据从字符串提取出来

In [ ]:
# 将字符串转换成数字
def data_adj(area_data, str):       
    if str in area_data :        
        return float(area_data[0 : area_data.find(str)])    
    else :        
        return None
# 处理房屋面积数据
house['meters'] = house['meters'].apply(data_adj,str = '平米')

## 获取总价前五的房源信息

In [ ]:
house.sort_values('price',ascending=False).head(5)

## 获取户型数量分布信息

In [ ]:
housetype = house['zone'].value_counts()
housetype.head(8).plot(kind='bar',x='zone',y='size', title='户型数量分布')
plt.legend(['数量']) 
plt.show()

## 关注人数最多5套房子

In [ ]:
htype = house.groupby('zone')['zone'].agg(len)
for x,y in htype.items():
    print(x.strip(), ": ", y)

## 户型和关注人数分布

In [ ]:
fig, ax1 = plt.subplots(1,1)    
type_interest_group = house['guanzhu'].groupby(house['housetype']).agg([('户型', 'count'), ('关注人数', 'sum')])    
#取户型>50的数据进行可视化
ti_sort = type_interest_group[type_interest_group['户型'] > 50].sort_values(by='户型')    
ti_sort.plot(kind='barh', alpha=0.7, grid=True, ax=ax1)    
plt.title('二手房户型和关注人数分布')    
plt.ylabel('户型') 
plt.show()

## 面积分布

In [ ]:
fig,ax2 = plt.subplots(1,1)    
area_level = [0, 50, 100, 150, 200, 250, 300, 500]    
label_level = ['小于50', '50-100', '100-150', '150-200', '200-250', '250-300', '300-350']    
area_cut = pd.cut(house['square'], area_level, labels=label_level)        
area_cut.value_counts().plot(kind='bar', rot=30, alpha=0.4, grid=True, fontsize='small', ax=ax2)    
plt.title('二手房面积分布')    
plt.xlabel('面积')    
plt.legend(['数量'])    
plt.show()

## 聚类分析

In [ ]:
# 缺失值处理:直接将缺失值去掉    
cluster_data = house[['guanzhu','square','totalPrice']].dropna()    
#将簇数设为3    
K_model = KMeans(n_clusters=3)    
alg = K_model.fit(cluster_data)    
'------聚类中心------'   
center = pd.DataFrame(alg.cluster_centers_, columns=['关注人数','面积','房价'])    
cluster_data['label'] = alg.labels_ 
center

## 北京市在售面积最小二手房

In [ ]:
house.sort_values('square').iloc[0,:]

## 北京市在售面积最大二手房

In [ ]:
house.sort_values('square',ascending=False).iloc[0,:]

## 各个行政区房源均价

In [ ]:
house_unitprice_perdistrict = house_detail.groupby('district').mean()['unitPrice']
house_unitprice_perdistrict.plot(kind='bar',x='district',y='unitPrice', title='各个行政区房源均价')
plt.legend(['均价']) 
plt.show()

## 各个区域房源数量排序

In [ ]:
bizcircle_count=house_detail.groupby('bizcircle').size().sort_values(ascending=False)
bizcircle_count.head(20).plot(kind='bar',x='bizcircle',y='size', title='各个区域房源数量分布')
plt.legend(['数量']) 
plt.show()

## 各个区域均价排序

In [ ]:
bizcircle_unitprice=house_detail.groupby('bizcircle').mean()['unitPrice'].sort_values(ascending=False)
bizcircle_unitprice.head(20).plot(kind='bar',x='bizcircle',y='unitPrice', title='各个区域均价分布')
plt.legend(['均价']) 
plt.show()

## 各个区域小区数量

In [ ]:
bizcircle_community=community.groupby('bizcircle')['title'].size().sort_values(ascending=False)
bizcircle_community.head(20).plot(kind='bar', x='bizcircle',y='size', title='各个区域小区数量分布')
plt.legend(['数量']) 
plt.show()

## 按小区均价排序

In [ ]:
community_unitprice = house.groupby('community').mean()['unitPrice'].sort_values(ascending=False)
community_unitprice.head(15).plot(kind='bar',x='community',y='unitPrice', title='各个小区均价分布')
plt.legend(['均价']) 
plt.show()